In [1]:
numDimensions = 300
maxSeqLength = 25
batchSize = 20
lstmUnits = 16
numClasses = 2
iterations = 100000

In [2]:
import numpy as np
wordsList = np.load('./training_data/wordsList.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('./training_data/wordVectors.npy')

In [3]:
import tensorflow as tf
tf.reset_default_graph()

# labels = tf.placeholder(tf.float32, [batchSize, numClasses], name="sentiment_labels")
# input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength], name="input_vector")

# data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32, name="3D_array")
# data = tf.nn.embedding_lookup(wordVectors,input_data, name="Integerizer")

# lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
# lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
# value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

# weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]), name="weight")
# bias = tf.Variable(tf.constant(0.1, shape=[numClasses]), name="bias")
# value = tf.transpose(value, [1, 0, 2], name="transposed_output")
# last = tf.gather(value, int(value.get_shape()[0]) - 1, name="last")
# prediction = (tf.matmul(last, weight) + bias)

# correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1), name="correct_prediction")
# accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32), name="accuracy")

with tf.name_scope('input_placeholders'):
    labels = tf.placeholder(tf.float32, [batchSize, numClasses], name="sentiment_labels")
    input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength], name="input_vector")
    
with tf.name_scope('word_embedding_array'):
    data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32, name="3D_array")
    data = tf.nn.embedding_lookup(wordVectors,input_data, name="Integerizer")

with tf.name_scope('LSTM_Params'):
    lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
    lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
    value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)
    
with tf.name_scope('LSTM_NN'):
    weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]), name="weight")
    bias = tf.Variable(tf.constant(0.1, shape=[numClasses]), name="bias")
    value = tf.transpose(value, [1, 0, 2], name="transposed_output")
    last = tf.gather(value, int(value.get_shape()[0]) - 1, name="last")
    prediction = (tf.matmul(last, weight) + bias)

with tf.name_scope('Prediction'):
    correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1), name="correct_prediction")
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32), name="accuracy")

with tf.name_scope('Optimizers'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels), name="Mean_Loss")
    optimizer = tf.train.AdamOptimizer(name="adam_optimizer").minimize(loss)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [4]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models/pretrained_lstm.ckpt-1000


In [5]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter,word in enumerate(split):
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix

In [6]:
inputText = input("Write your review: ")
# inputText = "i hated this product"

inputMatrix = getSentenceMatrix(inputText)
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
# predictedSentiment[0] represents output score for positive sentiment
# predictedSentiment[1] represents output score for negative sentiment

if (predictedSentiment[0] > predictedSentiment[1]):
    print ("Positive Sentiment")
else:
    print ("Negative Sentiment")

Write your review: I did not like it
Negative Sentiment


In [7]:
secondInputText = input("Write your next review: ")
secondInputMatrix = getSentenceMatrix(secondInputText)

predictedSentiment = sess.run(prediction, {input_data: secondInputMatrix})[0]
if (predictedSentiment[0] > predictedSentiment[1]):
    print ("Positive Sentiment")
else:
    print ("Negative Sentiment")

Write your next review: I really didn't like it
Negative Sentiment
